In [ ]:
import pandas as pd
import random
import time
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from urllib.parse import quote

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
df = pd.read_csv("Data/제주도식당_전처리.csv", index_col=False)

In [ ]:
df.head()

# 현재 없는데이터는 17001~19000까지의 데이터.

In [ ]:
df.count()

In [ ]:
df['사업장명']
scrapy_df2 = "제주 " + df['사업장명']
scrapy_df2

### 크롤링에 필요한 컬럼만 뽑아서 저장하기

In [ ]:
scrapy_df = df[['사업장명', "도로명전체주소"]]

In [ ]:
scrapy_df['도로명전체주소'] = scrapy_df['도로명전체주소'].str.split(',', n=1).str[0]
scrapy_df

In [ ]:
scrapy_df1 = scrapy_df['사업장명'] +' '+ scrapy_df['도로명전체주소']
scrapy_df1


In [ ]:
scrapy_df1
scrapy_df2

In [ ]:
# testData
scrapy_df1[17005:17006]

scrapy_df['사업장명'][17005:17006]



In [ ]:
# 크롬 드라이버 실행
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
url = f"https://map.naver.com/v5/search/"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
driver.get(url)

In [ ]:
# for i in range(17001, 17011):
#     url = f"https://map.naver.com/v5/search/{scrapy_df['사업장명'][i]} {scrapy_df['도로명전체주소'][i]}"
# # driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[1]/div/div[4]/a/span[2]/svg').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭

In [ ]:
scrapy_df['사업장명'][70:80]

# 리뷰추출 extract_review

In [ ]:
                            
# '''
# listsCount = len(lists)의 갯수
#     --------2, 3 개일 때 ---------
#     별점 블로그 방문자 모두 O listsCount == 3
#     별점 X , 블로그 방문자 O listsCount == 2
    
#     --------1 개일 때 ---------
#     별점 X 블로그 X, 방문자O listsCount == 1
# ========================================    
# 크롤링해올 review 종류 : 

# 구버전
#     listsCount == 1 일 때
#         1. 사진 O  더보기 O
        
#         2. 사진 O  더보기 X : 6 2 3    3
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[4]/div/div[3]/a/span
#         3. 사진 X  더보기 O : 6 2 3    3 / 6 2 3 1    2
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[3]/div/div[3]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[1]/div/div[2]/a/span
#             더보기
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[3]/div/div[3]/a/span[2]
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[1]/div/div[2]/a/span[2]
#         4. 사진 X  더보기 X : 6 2 3   2
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[6]/div/div[2]/a/span
        

#     listCount == 2,3일 때
#         1. 사진 O  더보기 O : 6 3 3 1   3
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[30]/div/div[3]/a/span
#             더보기 : 6 3 3 1   3
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[29]/div/div[3]/a

#         2. 사진 O  더보기 X : 6 3 3 1   3
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[27]/div/div[3]/a/span

#         3. 사진 X  더보기 O : 6 3 3 1   2
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[31]/div/div[2]/a/span
#             더보기 : 6 3 3 1  2
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[31]/div/div[2]/a

#         4. 사진 X  더보기 X : 6 3 3 1   2
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[26]/div/div[2]/a/span

# 신버전
#     listsCount == 1 일 때(옥성식당)
#         1. 사진 O  더보기 O : 6 3 3   4
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[1]/div/div[4]/a/span
#             더보기 : 6 3 3   4
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[4]/div/div[4]/a

#         2. 사진 O  더보기 X : 6 2 3 1  4  / 6 3 3   4
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[1]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[2]/div/div[4]/a/span

#         3. 사진 X  더보기 O : 6 2 3 1  3
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[3]/div/div[3]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[4]/div/div[3]/a/span
#             더보기
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[3]/div/div[3]/a
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[4]/div/div[3]/a
        
#         4. 사진 X  더보기 X : 6 2 3    3
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[1]/div/div[3]/a/span

#     listCount == 2,3일 때 (제주와 함께라면)
#         1. 사진 O  더보기 O : 6 3 3 1   4
#             더보기 : 6 3 3 1   4
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[4]/a
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[9]/div/div[4]/a
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[4]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[5]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[10]/div/div[4]/a/span
        
#         2. 사진 O  더보기 X : 6 3 3 1   4
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[1]/div/div[4]/a/span
        
#         3. 사진 X  더보기 O 
#             더보기 : 6 3 3 1    3
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[11]/div/div[3]/a
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[11]/div/div[3]/a/span
        
#         4. 사진 X  더보기 X : 6 3 3 1   3
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[6]/div/div[3]/a/span

# 사진이 있는 리뷰의 댓글과 사진이 없는 댓글의 경우 내용 자체가 다름. 


# 10개 넘어서 더보기 위해 '더보기'클릭
# listscount == 3
# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a

# listcount == 2
# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a

# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a


# '''
# def extract_review(listsCount: int):
#     time.sleep(random.uniform(2,2.5))
#     rev = []
#     # 미리 '더보기' 버튼을 두번 눌러 range(1,31)로 늘려놓는다.
#     if listsCount == 1:
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#             time.sleep(0.1)
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#             time.sleep(0.1)
#     else:
#         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         time.sleep(0.1)
#         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         time.sleep(0.1)

        
#     # 처음부터 끝까지 리뷰 추출
#     for i in range(1,31):
#         # listsCount == 1     
#         if listsCount == 1:
#             try: 
#                 # 텍스트 전체 볼 수 있게 클릭 구버전
#                 driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').send_keys(Keys.ENTER)
#                 time.sleep(0.1)
#                 # 리뷰 text
#                 comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').text
#                 rev.append(comment)
#         # 사진X

# 초기에는 크롤링 할 데이터를 전부 XPATH 로 진행하였으나 Xpath에는 예외가 너무 많음

# HTML, class로 크롤링 하기로 변경

In [ ]:
scrapy_df1[17005:17006]

In [134]:
# 연습용 ( 테스트용 )
t = random.randrange(1,2)
# 대기를 위한 WebDriverWait 객체 생성
wait = WebDriverWait(driver, t)
# 방문자 리뷰 클릭 후 그 이후 과정 (최신순 버튼 클릭 => 더보기 버튼 두번 클릭 )
def button_click_header_next():
    # 최신순 버튼 클릭
    late_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a')))
    print("최신순 버튼 클릭")
    late_button.click()
    time.sleep(0.5)

    # 더보기 버튼 두번 클릭 (더보기 버튼이있다면)
    try:
        more_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a')))
        if more_button:
            print("더보기 1 클릭")
            more_button.click()
            time.sleep(0.5)
            if more_button:
                more_button.click()
                print("더보기 2 클릭")
                time.sleep(0.5)
            else:
                print("더보기2 없음")
        else:
            print("더보기 없음")

        
    except Exception as e:
        pass

# 방문자 리뷰 클릭 버튼
def button_click_header():
    reviewSelector_ = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]')
    print(reviewSelector_.text)
    welcomeReviewButton = reviewSelector_.find_element(By.XPATH, './/a[contains(text(), "방문자리뷰")]')
    print("방문자 리뷰버튼 클릭 시도")
    welcomeReviewButton.click()
    time.sleep(1)
    button_click_header_next()

# 리뷰 추출30개 추출 함수
def review_extract(welcomeReviewCount: int):
    time.sleep(random.uniform(1.5,2.5))
    review = []
    review_text = ""
    if welcomeReviewCount > 30:
        welcomeReviewCount = 31
    

    for i in range(1, welcomeReviewCount):
        # detailreviewSelector = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]')
        # 감춰진 내용 보이게 하기.
        if driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[1]/div/div[3]/a'):
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[1]/div/div[3]/a').send_keys(Keys.ENTER)
        review_text = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]').text
        print(review_text)

for index, scrapyList in enumerate(scrapy_df1, start=17009):
    url1 = f"https://map.naver.com/v5/search/{scrapy_df1[index]}"
    url2 = f"https://map.naver.com/v5/search/{scrapy_df2[index]}"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
    driver.get(url1)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
    
    # 리뷰 추출30개 추출 함수
    
    

    button_click_header()
    review_extract(5)
    # # 첫 번째 요소가 존재하는지 확인 후 가져오기
    # first_elements = soup.select('span.PXMot a')
    # if first_elements:
    #     first_text = first_elements[0].text.strip()
    #     print(first_text)
    # else:
    #     print("첫 번째 리뷰가 존재하지 않습니다.")


    # # if reviewSelector:
    # #     for span in reviewSelector:
    # #         print(span.text)
    # # else:
    # #     print("리뷰를 찾을 수 없습니다.")
    # titleSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[1]')))
    # # starsSelector = soup.find('span', class_='PXMot LXIwF')
    
    # before_title = titleSelector.text
    # print(before_title)

    # # starsSelector = soup.select('.PXMot.LXIwF')
    # # starsSelector = driver.find_element(By.XPATH, '')
    
    # try:
    #     starsSelector = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'PXMot.LXIwF')))
    #     stars_ = starsSelector.text
    #     print({stars_})
    # except:
    #     print("별점없음")
    #         # 방문자리뷰, 블로그리뷰
    # try:
    #     # driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
    #     reviewSelector = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]').text
    #     print(f"{reviewSelector}찾음")
    #     # 정규식
    #     pattern1 = r'방문자리뷰 (\d+)블로그리뷰 (\d+)'
    #     pattern2 = r'방문자리뷰 (\d+[,\d]*)블로그리뷰 (\d+[,\d]*)'
        
        
    #     match1 = re.search(pattern1, reviewSelector)
    #     match2 = re.search(pattern2, reviewSelector)

    #     if match1:
    #         welcomeReviewCount_ = match1.group(1)
    #         blogReviewCount_ = match1.group(2)
    #         # welcomeReviewCount_ = int(welcomeReviewCount_.replace(',', '')) if ',' in welcomeReviewCount_ else int(welcomeReviewCount_)
    #         # blogReviewCount_ = int(blogReviewCount_.replace(',', '')) if ',' in blogReviewCount_ else int(blogReviewCount_)
    #         print("방문자리뷰 갯수:", welcomeReviewCount_)
    #         print("블로그리뷰 갯수:", blogReviewCount_)
    #     elif match2:
    #         welcomeReviewCount_ = match2.group(1)
    #         blogReviewCount_ = match2.group(2)
    #         # 쉼표(,)를 제거하고 정수로 변환
    #         welcomeReviewCount_ = int(welcomeReviewCount_.replace(',', '')) if ',' in welcomeReviewCount_ else int(welcomeReviewCount_)
    #         blogReviewCount_ = int(blogReviewCount_.replace(',', ''))  if ',' in blogReviewCount_ else int(blogReviewCount_)
    #         print("방문자리뷰 갯수:", welcomeReviewCount_)
    #         print("블로그리뷰 갯수:", blogReviewCount_)
    #     # elif match3:



    #     else:
    #         print("정보를 찾을 수 없습니다.")
    #     # print(f"{}찾음")
        
    # except Exception as e:
    #     print("리뷰 파싱 에러")
        
    #     # 방문자리뷰 클릭 => 최신순 클릭 => 더보기 클릭 => 더보기 클릭
    #     time.sleep(3)


별점
4.79방문자리뷰 12,187블로그리뷰 2,573
방문자 리뷰버튼 클릭 시도
최신순 버튼 클릭
더보기 1 클릭
더보기 2 클릭
연6198
리뷰 22사진 6
팔로우
이용 방법 예약 없이대기 시간 바로 입장목적 여행동행 연인·배우자
튀김 양도많고 맛있어요!! 창가쪽 자리 잘잡아서 바다보면서 먹으니까 분위기도좋네요
문어도 야들야들하고 설명도잘해주시고 
운좋으면 돌고래도볼수있고 너무좋은거같아요ㅋㅋ
양이 많아요음식이 맛있어요특별한 메뉴가 있어요매장이 청결해요뷰가 좋아요
반응 남기기
반응 남기기
방문일
3.15.금
2024년 3월 15일 금요일
1번째 방문
인증 수단
영수증
emk****
리뷰 1사진 4
팔로우
이용 방법 예약 없이대기 시간 바로 입장목적 여행동행 연인·배우자
육수넣기전 싱싱한 해산물을 찍었는데 보이는것 처럼 싱싱 하고 알찬 구성으로 여기 매니저님 친절하게 설명도 잘 해주 시고 밖에 보이는 뷰도 넘 좋아서 구경하다 보면 금새 ...
내용 더보기
재료가 신선해요+4
개의 리뷰가 더 있습니다
펼쳐보기
반응 남기기
반응 남기기
방문일
3.15.금
2024년 3월 15일 금요일
1번째 방문
인증 수단
영수증
lol****
리뷰 15사진 22
팔로우
이용 방법 예약 없이대기 시간 바로 입장목적 여행동행 혼자
여기 해물라면 정말 맛있습니다! 라면 해물 조합이 맛이 없을 수 있냐 할 수 있는데 (국물이 적거나, 짜거나...) 라면은 국 물인데 국물도 넉넉했고 면삶기도 적당 무엇보다 문...
내용 더보기
음식이 맛있어요+4
개의 리뷰가 더 있습니다
펼쳐보기
반응 남기기
반응 남기기
방문일
3.15.금
2024년 3월 15일 금요일
1번째 방문
인증 수단
영수증
강아지35
리뷰 3사진 4
팔로우
이용 방법 예약 없이대기 시간 바로 입장목적 여행동행 연인·배우자
사랑하는 사람이랑 제주 여행와서 드라이브 하다 들른곳인데 맛집이네요
재료가 신선해요+0
개의 리뷰가 더 있습니다
펼쳐보기
반응 남기기
반응 남기기
방문일
3.15.금
2024년 3월 15일 금요일
1번째 방문

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001052b0524 chromedriver + 3966244
1   chromedriver                        0x00000001052a8ab0 chromedriver + 3934896
2   chromedriver                        0x0000000104f2bda0 chromedriver + 277920
3   chromedriver                        0x0000000104f6e394 chromedriver + 549780
4   chromedriver                        0x0000000104fa6bf0 chromedriver + 781296
5   chromedriver                        0x0000000104f62fb0 chromedriver + 503728
6   chromedriver                        0x0000000104f63a28 chromedriver + 506408
7   chromedriver                        0x000000010527570c chromedriver + 3725068
8   chromedriver                        0x0000000105279c00 chromedriver + 3742720
9   chromedriver                        0x000000010525e1f4 chromedriver + 3629556
10  chromedriver                        0x000000010527a6fc chromedriver + 3745532
11  chromedriver                        0x000000010525156c chro

In [137]:
# # 리뷰를 추출해주는 함수
# def review_extract(listsCount: int):
#     time.sleep(random.uniform(2, 2.5))
#     rev = []  # 추출한 리뷰 저장
#     for i in range(1, 11): 
#         try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
#             if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
#                 driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#                 time.sleep(0.1)
#                 comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

#                 rev.append(comment)
#             else:
#                 driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#                 time.sleep(0.1)
#                 comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

#                 rev.append(comment)
#         except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
#             try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
#                 if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
#                     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                     time.sleep(0.1)
#                     comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                    
#                     rev.append(comment)
#                 else:
#                     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                     time.sleep(0.1)
#                     comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                    
#                     rev.append(comment)
#             except:
#                 rev.append("")

#     # 더 많은 리뷰가 있는지 확인하고, 있을 경우에는 추가 리뷰를 가져오기 위해 "더보기" 버튼 클릭
#     try:
#         # if listsCount == 1:
#         #     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         # else:
#         #     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)   
#         time.sleep(0.5)
#         for i in range(11, 21):
#             try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
#                 if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
#                     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#                     time.sleep(0.1)
#                     comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

#                     rev.append(comment)
#                 else:
#                     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#                     time.sleep(0.1)
#                     comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

#                     rev.append(comment)
#             except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
#                 try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
#                     if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
#                         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                         time.sleep(0.1)
#                         comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
#                         rev.append(comment)
#                     else:
#                         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                         time.sleep(0.1)
#                         comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
#                         rev.append(comment)
#                 except:
#                     rev.append("")

#     except:
#         print('더 이상 리뷰가 없음')
    
#     try:
#         # if listsCount == 1:
#         #     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         # else:
#         #     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)   
#         time.sleep(0.5)
#         for i in range(21, 31):
#             try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
#                 if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
#                     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#                     time.sleep(0.1)
#                     comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

#                     rev.append(comment)
#                 else:
#                     driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
#                     time.sleep(0.1)
#                     comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰

#                     rev.append(comment)
#             except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
#                 try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
#                     if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
#                         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                         time.sleep(0.1)
#                         comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
#                         rev.append(comment)
#                     else:
#                         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
#                         time.sleep(0.1)
#                         comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        
#                         rev.append(comment)

#                 except:
#                     rev.append("")

#     except:
#         print('더 이상 리뷰가 없음')
    
    
#     return rev




# 실제코드

In [151]:
scrapy_df1[17005:17006]

# ===== 크롤링 설정 =====
idx = 0 # 크롤링 총 갯수
failed_title_idx = 0 # title 크롤링 실패시 idx 1 증가
failed_stars_idx = 0 # stars 크롤링 실패시 idx 1 증가
failed_Review_idx = 0

header_count = 0 # header파트에 (별점, 방문자리뷰, 블로그리뷰) == 0 or 1 or 2 or 3 

t = random.randrange(1,2)
# 대기를 위한 WebDriverWait 객체 생성
wait = WebDriverWait(driver, t)


# 크롤링한 데이터 담길곳
scrapy_header_data = pd.DataFrame(columns=['Title', 'titleDetail', 'stars', 'welcomReviewCount','blogReviewCount'])

# 크롤링중 에러발생 데이터 담길 곳. 
'''
1. notSearche : 데이터를 찾지 못하였을 때
2. Title : 제목관련 데이터 찾지 못했을 때
3. stars : 별점 관련 데이터 없을 때
4. ReviewCount : 리뷰데이터 없을 때
'''
scrapy_error_data = pd.DataFrame(columns=['notSearch', 'Title', 'stars', 'ReviewCount'])

'''
크롤링 기준: dfList1 검색 
if dfList1 can not find iframe :
    use dfList2
'''

# ===== 크롤링 해올 데이터 담길 배열 ======
title = [] #이름 담길 배열 (div class = 'Fc1rA) // EX: 마야블루
titleDetail = [] #이름 옆에 분류 담길 배열 (div class = 'DJJvD) // EX: 패션잡화
stars = [] #별점 담길 배열 (별점은 있을 수도, 없을 수도 있음. 예외처리필요)
welcomeReviewCount = [] #방문자 리뷰 (PXMot)
blogReviewCount = [] #블로그 리뷰 (PXMot)
reviewData = [] #방문자 리뷰 상세 내용

# ===== Error Log 담길 배열 =====
error_notSearch = [] # = 검색결과 없음
error_title = []
error_stars = []
error_ReviewCount = []

def add_error_data(error_type, index, title):
    if error_type == 'notSearch':
        error_notSearch.append({"error_index": index, 'error_name': title})
    elif error_type == 'title':
        error_title.append({"error_index": index, "error_title": title})
    elif error_type == 'stars':
        error_stars.append({"error_index": index, "error_title": title})
    elif error_type == 'ReviewCount':
        error_ReviewCount.append({"error_index": index, "error_title": title})
        
# 방문자 리뷰 클릭 후 그 이후 과정 (최신순 버튼 클릭 => 더보기 버튼 두번 클릭 )
def button_click_header_next():
    # 최신순 버튼 클릭
    late_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a')))
    print("최신순 버튼 클릭")
    late_button.click()
    time.sleep(0.5)

    # 더보기 버튼 두번 클릭 (더보기 버튼이있다면)
    try:
        more_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a')))
        if more_button:
            print("더보기 1 클릭")
            more_button.click()
            time.sleep(0.5)
            if more_button:
                more_button.click()
                print("더보기 2 클릭")
                time.sleep(0.5)
            else:
                print("더보기2 없음")
        else:
            print("더보기 없음")

        
    except Exception as e:
        pass

# 방문자 리뷰 클릭 버튼
def button_click_header():
    reviewSelector_ = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]')
    # print(reviewSelector_.text)
    welcomeReviewButton = reviewSelector_.find_element(By.XPATH, './/a[contains(text(), "방문자리뷰")]')
    welcomeReviewButton.click()
    time.sleep(1)
    print("방문자 리뷰버튼 클릭 완료")
    button_click_header_next()

# 리뷰 추출30개 추출 함수
def review_extract(welcomeReviewCount, lists):
    time.sleep(random.uniform(1.5,2.5))
    

    print(f"방문자리뷰갯수 = {welcomeReviewCount}")
    print(f"PXMot갯수 = {lists}")
    
    if welcomeReviewCount > 30:
        welcomeReviewCount = 31
    
    # lists : PXMot의 갯수 => PXMot의 갯수는 어덯게 이용하는가?
    # welcomeReviewCount : 방문자 리뷰의 갯수 이걸로 for문 만듬
    
    # for i in range(1, welcomeReviewCount):
    #     review_text = ""
        # 구버전
            # lists == 1 일 때 
        
        # 끝값에 도달할 때까지 반복
    review_text = ""
    review_texts = []
    i = 1
    while True:
        try:
            review_text = ""
            # 리뷰 텍스트 가져오기
            review_text = driver.find_element(By.XPATH, f'#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div.place_section_content > ul > li:nth-child({i}) > div > div.vg7Fp > a > span').text
            print(review_text)
            review_texts.append(review_text)

            i += 1
        except NoSuchElementException:
            # 끝값에 도달하면 반복 종료
            break
        # try:
        #     if lists == 1:
        #         # 구버전이고 lists == 1일 때
        #         try:
        #             review_text = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div/div[3]/a/span').text
        #             print(review_text)
        #         # 신버전이고 lists == 1일 때
        #         except:
        #             review_text = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{i}]/div/div[3]/a/span').text
        #         print(f"reviewText = {review_text}")
        # except NoSuchElementException:
        #     review_text = "리뷰 없음"
        #     print("리뷰 없음")
        # 결과 출력
        for index, text in enumerate(review_texts, start=1):
            print(f"리뷰 {index}: {text}")

    return review_texts
    


'''
====================================================================
========================= 본코드Start ================================
====================================================================
'''
for index, scrapyList in enumerate(scrapy_df1, start=1):
    url1 = f"https://map.naver.com/v5/search/{scrapy_df1[index]}"
    url2 = f"https://map.naver.com/v5/search/{scrapy_df2[index]}"
    driver.get(url1)
    idx += 1
    print(f"====================번호{idx}==================")
    time.sleep(3)
    # ====================Fucntion=========================
    # 버튼클릭 함수(방문자 리뷰 => 최신순 => 더보기 => 더보기)
    
    '''
    try:
        url1으로 도전: 사업장명 + 주소
    except:
        url2로 도전: 제주 + 사업장명

    '''
    # url1으로 들어가 항목이 있는지 체크, 별점, 방문자리뷰, 블로그리뷰 갯수만 크롤링
    try:
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동   
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(3)
        lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
        lists = len(lists)

        # title, titleDetail
        try:
            
            # title, titleDetail Selector선언
            titleSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[1]')))
            titleDetailSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[2]')))

            # title, titleDetail가져오기
            # title
            before_title = titleSelector.text
            print(f"title : {before_title}")
            title.append(before_title)
            # titleDetail   
            before_titleDetail = titleDetailSelector.text
            print(f"상세분류 : {before_titleDetail}")
            titleDetail.append(before_titleDetail)
        except Exception as e:
            add_error_data('title', index, scrapyList)
            failed_title_idx += 1
            print("===========title 크롤링 실패")

        # 별점
        try:
            starsSelector = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'PXMot.LXIwF')))
            
            stars_ = starsSelector.text
            # 텍스트에서 "별점" 제거하고 공백을 기준으로 분리하여 첫 번째 요소를 선택하여 숫자값만 저장
            stars_ = stars_.split()[1]
            stars.append(stars_)
            print(f"별점 : {stars_}")
                
        except Exception as e:
            add_error_data('stars', index, scrapyList)
            stars.append("X")
            print("별점 없음")
            failed_stars_idx += 1

        # 방문자리뷰, 블로그리뷰
        welcomeReviewCount_ = ""
        blogReviewCount_ = ""
        try:
            reviewSelector = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]').text
        
            # 정규식
            pattern1 = r'방문자리뷰 (\d+)블로그리뷰 (\d+)'
            pattern2 = r'방문자리뷰 (\d+[,\d]*)블로그리뷰 (\d+[,\d]*)'
            pattern3 = r'방문자리뷰 (\d+[,\d]*)'
            pattern4 = r'블로그리뷰 (\d+[,\d]*)'
            
            
            match1 = re.search(pattern1, reviewSelector)
            match2 = re.search(pattern2, reviewSelector)
            match3 = re.search(pattern3, reviewSelector)
            match4 = re.search(pattern4, reviewSelector)

            if match1:
                print('match1')
                welcomeReviewCount_ = match1.group(1)
                blogReviewCount_ = match1.group(2)
                # welcomeReviewCount_ = int(welcomeReviewCount_.replace(',', '')) if ',' in welcomeReviewCount_ else int(welcomeReviewCount_)
                # blogReviewCount_ = int(blogReviewCount_.replace(',', '')) if ',' in blogReviewCount_ else int(blogReviewCount_)
            elif match2:
                print('match2')
                welcomeReviewCount_ = match2.group(1)
                blogReviewCount_ = match2.group(2)
                # 쉼표(,)를 제거하고 정수로 변환
                welcomeReviewCount_ = int(welcomeReviewCount_.replace(',', '')) if ',' in welcomeReviewCount_ else int(welcomeReviewCount_)
                blogReviewCount_ = int(blogReviewCount_.replace(',', ''))  if ',' in blogReviewCount_ else int(blogReviewCount_)
            elif match3:
                print('match3')
                welcomeReviewCount_ = match3.group(1)
                welcomeReviewCount_ = int(welcomeReviewCount_)
                blogReviewCount_ = "블로그 리뷰 없음"
            elif match4:
                print('match4')
                welcomeReviewCount_ = "방문자 리뷰 없음"
                blogReviewCount_ = match4.group(1)
                blogReviewCount_ = int(blogReviewCount_)
            else:
                welcomeReviewCount_ = "방문자 리뷰 없음"
                blogReviewCount_ = "블로그 리뷰 없음"
            print("방문자리뷰 갯수:", welcomeReviewCount_)
            print("블로그리뷰 갯수:", blogReviewCount_)
            welcomeReviewCount.append(welcomeReviewCount_)
            blogReviewCount.append(blogReviewCount_)
            time.sleep(1)

            #방문자 버튼 클릭
            button_click_header()

            # review에 추출한 리뷰데이터들 넣음
            review = review_extract(welcomeReviewCount_, lists)
            reviewData.append(review)

        except Exception as e:
            add_error_data('ReviewCount', index, scrapyList)
            print("리뷰 파싱 에러1")
            failed_Review_idx += 1

    

    # ==========================================================================================================================================================
    # URL1 감지 불가 URL2로 시도.
    # ==========================================================================================================================================================
    except NoSuchElementException:
        print(f"url2로 검색 시도")
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="searchIframe"]'))
        time.sleep(2)
        driver.get(url2)
        try:
            driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
            print(f"{index}업체 있음")
            time.sleep(2)

            

            # title, titleDetail
            try:
                # title, titleDetail
                titleSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[1]')))
                titleDetailSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[2]')))

                # title, titleDetail가져오기
                # title
                before_title = titleSelector.text
                print(f"title : {before_title}")
                title.append(before_title)
                # titleDetail   
                before_titleDetail = titleDetailSelector.text
                print(f"상세분류 : {before_titleDetail}")
                titleDetail.append(before_titleDetail)
                time.sleep(1)
            except Exception as e:
                add_error_data('title', index, scrapyList)
                failed_title_idx += 1
                print("===========title 크롤링 실패")

            # 별점
            try:
                starsSelector = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'PXMot.LXIwF')))
                
                stars_ = starsSelector.text
                # 텍스트에서 "별점" 제거하고 공백을 기준으로 분리하여 첫 번째 요소를 선택하여 숫자값만 저장
                stars_ = stars_.split()[1]
                stars.append(stars_)
                print(f"별점 : {stars_}")
                    
            except Exception as e:
                add_error_data('stars', index, scrapyList)
                stars.append("X")
                print("별점 없음")
                failed_stars_idx += 1

            # 방문자리뷰, 블로그리뷰
            welcomeReviewCount_ = ""
            blogReviewCount_ = ""
            try:
                reviewSelector = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]').text
                
                

                # 정규식
                pattern1 = r'방문자리뷰 (\d+)블로그리뷰 (\d+)'
                pattern2 = r'방문자리뷰 (\d+[,\d]*)블로그리뷰 (\d+[,\d]*)'
                pattern3 = r'방문자리뷰 (\d+[,\d]*)'
                pattern4 = r'블로그리뷰 (\d+[,\d]*)'
                
                
                match1 = re.search(pattern1, reviewSelector)
                match2 = re.search(pattern2, reviewSelector)
                match3 = re.search(pattern3, reviewSelector)
                match4 = re.search(pattern4, reviewSelector)

                if match1:
                    print('match1')
                    welcomeReviewCount_ = match1.group(1)
                    blogReviewCount_ = match1.group(2)
                    # welcomeReviewCount_ = int(welcomeReviewCount_.replace(',', '')) if ',' in welcomeReviewCount_ else int(welcomeReviewCount_)
                    # blogReviewCount_ = int(blogReviewCount_.replace(',', '')) if ',' in blogReviewCount_ else int(blogReviewCount_)
                elif match2:
                    print('match2')
                    welcomeReviewCount_ = match2.group(1)
                    blogReviewCount_ = match2.group(2)
                    # 쉼표(,)를 제거하고 정수로 변환
                    welcomeReviewCount_ = int(welcomeReviewCount_.replace(',', '')) if ',' in welcomeReviewCount_ else int(welcomeReviewCount_)
                    blogReviewCount_ = int(blogReviewCount_.replace(',', ''))  if ',' in blogReviewCount_ else int(blogReviewCount_)
                elif match3:
                    print('match3')
                    welcomeReviewCount_ = match3.group(1)
                    welcomeReviewCount_ = int(welcomeReviewCount_)
                    blogReviewCount_ = "블로그 리뷰 없음"
                elif match4:
                    print('match4')
                    welcomeReviewCount_ = "방문자 리뷰 없음"
                    blogReviewCount_ = match4.group(1)
                    blogReviewCount_ = int(blogReviewCount_)
                else:
                    welcomeReviewCount_ = "방문자 리뷰 없음"
                    blogReviewCount_ = "블로그 리뷰 없음"
                print("방문자리뷰 갯수:", welcomeReviewCount_)
                print("블로그리뷰 갯수:", blogReviewCount_)
                welcomeReviewCount.append(welcomeReviewCount_)
                blogReviewCount.append(blogReviewCount_)
                time.sleep(1)
                button_click_header()
                review = review_extract(welcomeReviewCount_, lists)
                reviewData.append(review)

            except Exception as e:
                add_error_data('ReviewCount', index, scrapyList)
                print("리뷰 파싱 에러2")
                failed_Review_idx += 1
        

        except NoSuchElementException:
            print(f"index = {index}")
            print(f"{scrapy_df1[index]} -  조건에 맞는 업체가 없습니다.")
            error_notSearch.append({"error_index": index, "error_title": scrapy_df1[index]})
            continue



    

====================번호1==================
title : 에릭스에스프레소
상세분류 : 카페,디저트
별점 : 4.49
match1
방문자리뷰 갯수: 97
블로그리뷰 갯수: 44
방문자 리뷰버튼 클릭 완료
최신순 버튼 클릭
더보기 1 클릭
더보기 2 클릭
방문자리뷰갯수 = 97
PXMot갯수 = 3
리뷰 파싱 에러1
====================번호2==================
title : 정가네 일품순두부 한림점
상세분류 : 두부요리
별점 : 4.35
match1
방문자리뷰 갯수: 432
블로그리뷰 갯수: 44
방문자 리뷰버튼 클릭 완료
최신순 버튼 클릭
더보기 1 클릭
더보기 2 클릭
방문자리뷰갯수 = 432
PXMot갯수 = 3
리뷰 파싱 에러1
====================번호3==================


KeyboardInterrupt: 

In [144]:
welcomeReviewCount
blogReviewCount
add_error_data


<function __main__.add_error_data(error_type, index, title)>

In [ ]:
count = 0 # 성공한 크롤링 카운트
failedCount = 0 # 실패한 크롤링 카운트
# scrapy_df1 : '사업장명' + '주소'
# scrapy_df2 : '제주' + '사업장명'

crawlingReviewData = [] # 식당 리뷰를 저장할 리스트
crawlingImageData = [] # 식당 이미지를 저장할 리스트
crawlingCategoryData = [] # 식당 업종분류를 저장할 리스트
crawlingVisitCountData = [] # 식당 방문자리뷰 수를 저장할 리스트
crawlingBlogCountData = [] # 식당 블로그리뷰 수를 저장할 리스트
crawlingRatingData = [] # 식당 평점을 저장할 리스트
errorResult = ""

result_df = pd.DataFrame()# 결과저장
# 테스트 데이터1 :(별점O, 방문자O, 블로그O) 제주와 함께라면 : listcount ==  3
# 테스트 데이터2 :(별점X, 방문자O, 블로그O) 만사옥게 : listcount == 2
# 테스트 데이터3 :(별점X, 방문자O, 블로그X) 옥성식당 : listcount == 1
for index, scrapyList in enumerate(scrapy_df1[17005:17006], start = 17005):
    url1 = f"https://map.naver.com/v5/search/{scrapy_df1[index]}"
    url2 = f"https://map.naver.com/v5/search/{scrapy_df2[index]}"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
    driver.get(url1)
    print(f"=================url1 {index}시도=================")
    time.sleep(random.uniform(2.0, 2.5))
    try: 
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(random.uniform(1.5, 2.0))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print(f'"=================url1 {index}찾음=================')
    except NoSuchElementException as e:
        driver.get(url2)
        print(f"=================url2 {index}시도=================")
        time.sleep(random.uniform(2.0, 2.5))
        try:
            driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
            time.sleep(random.uniform(1.5, 2.0))
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            print("=================url2 {index}찾음=================")
        except NoSuchElementException as e:
            print(f"{index} 검색결과를 찾을 수 없음.")
            continue

        # 이미지 3장 뽑아서 저장하기
        images = []  # 이미지 링크를 저장할 리스트
        for i in range(1, 4):
            image_element = soup.select_one(f"#ibu_{i}")  # ibu_1 ~ 3 이런식으로 이미지의 id가 갯수별로 형성됨
            if not image_element:
                continue  # 이미지 요소가 없으면 for 루프 종료
            style = image_element["style"]
            background_image = style.split("url(")[1].split(")")[0]
            images.append(background_image)  # 이미지 링크를 리스트에 추가

        # 음식 카테고리 뽑기 (버튼이라서 CSS_SELECTOR를 써야함)
        foodCategory = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.DJJvD').text


        # 리뷰 탭을 선택하기 전에 버튼 갯수 파악하기 (별점 4.7, 방문자리뷰 100, 블로그리뷰 120) 이런식으로 된곳
        lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
        errorResult = "검색 결과 없음"
        '''
        ratingText = 별점
        visitText = 방문자리뷰 갯수
        blogText = 블로그리뷰 갯수
        '''
        if len(lists) == 3: # (별점,블로그,방문자 있는 경우) 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
            # 별점 , 방문자리뷰 갯수, 블로그리뷰 갯수 긁어오기
            ratingText = soup.select_one('.PXMot.LXIwF').text # 별점의 class = .PXMot.LXIwF
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a').text
            

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 2: # (별점이 없는 경우) 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
            # 리뷰 수 긁어오기                                 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            ratingText = "x"

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 1: # 블로그 리뷰가 없는 경우 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a').text
            blogText = "블로그리뷰 0"
            ratingText = "x"

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1) 
        else:
            errorResult = "크롤링 불가"
# 여기까지가 이제 방문자리뷰 들어가서 최신순 클릭한 상태.

# 리뷰 추출 함수 소환
        review = extract_review(len(lists))
########
        crawlingReviewData.append(review)
        crawlingImageData.append(images)
        crawlingCategoryData.append(foodCategory)
        crawlingRatingData.append(ratingText.replace("별점", ""))
        crawlingVisitCountData.append(int(re.sub(r'\D', '', visitText)))  # 정규표현식을 사용하여 숫자만 추출
        crawlingBlogCountData.append(int(re.sub(r'\D', '', blogText)))
        count += 1
        print("----------------------------------")
        print(f'크롤링 성공! 성공: {count}, 실패: {failedCount}')
        ## 별점이 있고 없고에 따라 공백 데이터 추가해주는거 필요
    except:
        crawlingReviewData.append('x')
        crawlingImageData.append('x')
        crawlingCategoryData.append('x')
        crawlingRatingData.append('x')
        crawlingVisitCountData.append("x")
        crawlingBlogCountData.append("x")
        failedCount += 1
        print("----------------------------------")
        print(f'크롤링 실패! 성공: {count}, 실패: {failedCount}, 이유: {errorResult}')

    result_df = pd.DataFrame({'foodCategory': crawlingCategoryData, 'images': crawlingImageData, 'rating': crawlingRatingData, 'visitReviewCount': crawlingVisitCountData, 'blogReviewCount': crawlingBlogCountData,'review': crawlingReviewData})


In [ ]:
result_df

In [ ]:
foodCategory

In [ ]:
# 최신순 버튼 클릭 못하네
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
                          #    //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a 
                                                                    #  ㄴ기존 코드랑 달라진 부분

In [ ]:
###  본인작성 리뷰만 가져오자. 자동작성 리뷰는 긍정뿐이 없음.
### 사진 x
# 2번리뷰 (사진 X, 본인작성 리뷰, 상세한 리뷰) - 상세한 리뷰 ( 이용방법, 대기시간, 목적, 동행 등을 입력한 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[3]/a/span').text 
# 6번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 ) - 기존리뷰
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[6]/div/div[2]/a/span').text
# 8번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[8]/div/div[2]/a/span').text
# 9번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[9]/div/div[2]/a/span').text
# 10번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[10]/div/div[2]/a/span').text

### 사진 o
# 3번 리뷰 (사진 O, 본인작성 리뷰)
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[3]/div/div[4]/a/span').text
# 4번 리뷰 (사진 O, 본인작성 리뷰)
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[4]/div/div[4]/a/span').text


# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text



In [ ]:
# 10번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[10]/div/div[2]/a').text

In [ ]:

# driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a/span

driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a/span').send_keys(Keys.ENTER)


In [ ]:
lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
errorResult = "검색 결과 없음"
if len(lists) == 3: # (별점이 있는 경우) 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
    # 별점 수, 리뷰 수 긁어오기
    ratingText = soup.select_one('.PXMot.LXIwF').text # 별점의 class = .PXMot.LXIwF
    visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
    blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a').text

    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
    time.sleep(1)
elif len(lists) == 2: # (별점이 없는 경우) 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
    # 리뷰 수 긁어오기                                 
    visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
    blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
    ratingText = "x"

    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
    time.sleep(1)
elif len(lists) == 1: # 블로그 리뷰가 없는 경우 
    visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a').text
    blogText = "블로그리뷰 0"
    ratingText = "x"

    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
    time.sleep(1) 
else:
    errorResult = "크롤링 불가"
###########
# 위 내용이 최신순까지 클릭한 상태.
# 아래 내용은 extract_review를 정의하고 review만 가져오는 함수.
###########    
'''
크롤링해올 review 종류 : 
구버전
    1. 사진 O 내용 O 더보기 O
    2. 사진 O 내용 O 더보기 X
    3. 사진 X 내용 O 더보기 O 
    4. 사진 X 내용 O 더보기 X
신버전
    1. 사진 O 내용 O 더보기 O
    2. 사진 O 내용 O 더보기 X
    3. 사진 X 내용 O 더보기 O
    4. 사진 X 내용 O 더보기 X

사진이 있는 리뷰의 댓글과 사진이 없는 댓글의 경우 내용 자체가 다름. 
더보기버튼은 if문으로 뚫자.
for

'''

def extract_review(listsCount: int):
            time.sleep(random.uniform(2,2.5))
            rev = []
            for i in range(1,11):
                try:
                    # 사진 O 구버전
                    if listsCount == 1:
                        # 텍스트 전체 볼 수 있게 클릭
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').send_keys(Keys.ENTER)
                        time.sleep(0.1)
                        # 리뷰 text
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').text
                        rev.append(comment)
                except:
                     
                    # 사진X


review = extract_review(len(lists))